Dijkstra's Shortest Path Algorithm
-------------------------------------------------

First define the test map as an array. Each entry in the array represents a grid cell on the map.

In [1]:
import numpy as np

# test_map1 = np.array([
#               [1, 1, 1, 1, 1, 1, 1, 1],
#               [1, 0, 0, 0, 0, 0, 0, 1],
#               [1, 0, 0, 0, 0, 0, 0, 1],
#               [1, 0, 0, 0, 0, 0, 0, 1],
#               [1, 0, 0, 0, 0, 0, 0, 1],
#               [1, 0, 0, 0, 0, 0, 0, 1],
#               [1, 0, 0, 0, 0, 0, 0, 1],
#               [1, 0, 0, 0, 0, 0, 0, 1],
#               [1, 0, 0, 0, 0, 0, 0, 1],
#               [1, 1, 1, 1, 1, 1, 1, 1]])

test_map1 = np.array([
             [0, 0, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0],
             [1, 1, 1, 1, 1, 1, 1, 1],
             [1, 0, 0, 1, 1, 0, 0, 1],
             [1, 0, 0, 1, 1, 0, 0, 1],
             [1, 0, 0, 1, 1, 0, 0, 1],
             [1, 0, 0, 0, 0, 0, 0, 1],
             [1, 0, 0, 0, 0, 0, 0, 1],
             [1, 1, 1, 1, 1, 1, 1, 1]])
occupancy_map = test_map1

Define the spacing in the x (horizontal) direction between adjacent columns and the y (vertical) direction between adjacent rows. The positive x direction is to the right and the positive y direction is **downward**.

In [2]:
# x_spacing1 = 0.13
# y_spacing1 = 0.2
x_spacing1 = 0.2
y_spacing1 = 0.2
x_spacing = x_spacing1
y_spacing = y_spacing1

Define the start and goal positions.

In [3]:
# start1 = np.array([[0.3], [0.3], [0]])
# goal1 = np.array([[0.6], [1], [0]])
start1 = np.array([[0.5], [1.0], [1.5707963267948966]])
goal1 = np.array([[1.1], [0.9], [-1.5707963267948966]])
start = start1
goal = goal1

Keep track of the state of each grid cell by populating a new array. A value of 1 represents a free cell and a value of 2 represents an obstacle cell.

In [4]:
state_map = np.zeros(occupancy_map.shape) # state_map.shape = (8, 8)
state_map[occupancy_map == 0] = 1
state_map[occupancy_map == 1] = 2
state_map

array([[1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.],
       [2., 2., 2., 2., 2., 2., 2., 2.],
       [2., 1., 1., 2., 2., 1., 1., 2.],
       [2., 1., 1., 2., 2., 1., 1., 2.],
       [2., 1., 1., 2., 2., 1., 1., 2.],
       [2., 1., 1., 1., 1., 1., 1., 2.],
       [2., 1., 1., 1., 1., 1., 1., 2.],
       [2., 2., 2., 2., 2., 2., 2., 2.]])

Next we need to determine the node index of the starting position in both the x and y directions. A node is defined to be at the center of a grid cell. However, it is possible (and most likely the case) that the starting position is not exactly lined up with node and we need to find out which one of the four cases is true:

1. Start position is between two nodes in the x-direction.
2. Start position is between two nodes in the y-direction.
3. Start position is between two nodes in the x and y directions.
4. Start position is exactly on top of a node.

Let's define a grid in the x and y directions to check for each of the cases. Array entries represent the position of the left edge of the grid cell (not the node).

In [5]:
x_grid = np.arange(state_map.shape[1])*x_spacing
y_grid = np.arange(state_map.shape[0])*y_spacing
print("x_grid = " + str(x_grid))
print("y_grid = " + str(y_grid))

x_grid = [0.  0.2 0.4 0.6 0.8 1.  1.2 1.4]
y_grid = [0.  0.2 0.4 0.6 0.8 1.  1.2 1.4 1.6 1.8]


Check if starting position is in between two nodes in the x-direction. This means the starting x-coordinate is on the edge of a grid cell.

In [6]:
tol = 1e-6
startBtw2xNodes = False
startBtw2yNodes = False
if np.abs(np.mod(start[0].item(), x_spacing) - x_spacing) < tol: # starting position exactly on edge of grid cell
    jStart = np.asscalar(np.searchsorted(x_grid, start[0])) # find index in x_grid that corresponds to starting x-coordinate
    startBtw2xNodes = True
else:
    jStart = np.asscalar(np.searchsorted(x_grid, start[0])) - 1 # find index in x_grid to the left of starting x-coordinate
    
print("startBtw2xNodes = " + str(startBtw2xNodes)) # start = (0.3, 0.3) so should be false
print("jStart = " + str(jStart))

startBtw2xNodes = False
jStart = 2


Repeat the same procedure for y-direction. Check if starting position is in between two nodes in the y-direction. This means the starting y-coordinate is on the edge of a grid cell.

In [7]:
if np.abs(np.mod(start[1].item(), y_spacing) - y_spacing) < tol: # starting position exactly on edge of grid cell
    iStart = np.asscalar(np.searchsorted(y_grid, start[1])) # find index in x_grid that corresponds to starting x-coordinate
    startBtw2yNodes = True
else:
    iStart = np.asscalar(np.searchsorted(y_grid, start[1])) - 1 # find index in x_grid to the left of starting x-coordinate
    
print("startBtw2yNodes = " + str(startBtw2yNodes)) # start = (0.3, 0.3) so should be false
print("iStart = " + str(iStart))

startBtw2yNodes = True
iStart = 5


In [8]:
print(start[1].item())
print(y_spacing)
print(np.abs(-1))
np.mod(start[1].item(), y_spacing)

1.0
0.2
1


0.19999999999999996

Store all of the possible starting nodes in a list. If the starting position is exactly in between two nodes in the x and/or y-directions then this increases the number of possible starting nodes. If the starting position is exactly on a node, then the starting node is the starting position.

In [9]:
if startBtw2xNodes and startBtw2yNodes:
    possibleStartNodes = [(iStart,jStart),(iStart,jStart-1),(iStart-1,jStart),(iStart-1,jStart-1)]
elif startBtw2xNodes:
    possibleStartNodes = [(iStart,jStart),(iStart,jStart-1)]
elif startBtw2yNodes:
    possibleStartNodes = [(iStart,jStart),(iStart-1,jStart)]
else:
    possibleStartNodes = [(iStart,jStart)]
    
print("possibleStartNodes =")
print('\n'.join(map(str, possibleStartNodes)))

possibleStartNodes =
(5, 2)
(4, 2)


In [10]:
# Initialize start node
start_node = (iStart, jStart)

# Initialize linear node position in row-major order
start_linInd = np.ravel_multi_index(start_node, state_map.shape)

print("start_node = " + str(start_node))
print("start_linInd = " + str(start_linInd))

start_node = (5, 2)
start_linInd = 42


Repeat the same procedure for the goal position.

In [11]:
goalBtw2xNodes = False
goalBtw2yNodes = False
if np.abs(np.mod(goal[0].item(), x_spacing) - x_spacing) < tol: # starting position exactly on edge of grid cell
    jGoal = np.asscalar(np.searchsorted(x_grid, goal[0])) # find index in x_grid that corresponds to starting x-coordinate
    goalBtw2xNodes = True
else:
    jGoal = np.asscalar(np.searchsorted(x_grid, goal[0])) - 1 # find index in x_grid to the left of starting x-coordinate

if np.abs(np.mod(goal[0].item(), y_spacing) - y_spacing) < tol: # starting position exactly on edge of grid cell
    iGoal = np.asscalar(np.searchsorted(y_grid, goal[1])) # find index in x_grid that corresponds to starting x-coordinate
    goalBtw2yNodes = True
else:
    iGoal = np.asscalar(np.searchsorted(y_grid, goal[1])) - 1 # find index in x_grid to the left of starting x-coordinate

if goalBtw2xNodes and goalBtw2yNodes:
    possibleGoalNodes = [(iGoal,jGoal),(iGoal,jGoal-1),(iGoal-1,jGoal),(iGoal-1,jGoal-1)]
elif goalBtw2xNodes:
    possibleGoalNodes = [(iGoal,jGoal),(iGoal,jGoal-1)]
elif goalBtw2yNodes:
    possibleGoalNodes = [(iGoal,jGoal),(iGoal-1,jGoal)]
else:
    possibleGoalNodes = [(iGoal,jGoal)]

# Initialize goal node
goal_node = (iGoal, jGoal)

# Initialize linear node position in row-major order
goal_linInd = np.ravel_multi_index(goal_node, state_map.shape)

print("goalBtw2xNodes = " + str(goalBtw2xNodes))
print("jGoal = " + str(jGoal))
print("goalBtw2yNodes = " + str(goalBtw2yNodes))
print("iGoal = " + str(iGoal))

goalBtw2xNodes = False
jGoal = 5
goalBtw2yNodes = False
iGoal = 4


Initialize 2-D array where each cell holds the distance from start node.

In [12]:
distanceFromStart = np.full(state_map.shape, np.inf) # 10x10 array of inf

Initialize 2-D array where each cell holds the index of its parent.

In [13]:
parent = np.zeros(state_map.shape)

In [14]:
# Initialize distance from start at the start node to zero.
distanceFromStart[start_node] = 0

In [15]:
# Begin while loop

# Update state map with start node
state_map[start_node] = 5

while True

    # Find node with minimum distance from start
    min_dist = np.min(distanceFromStart)
    current_linInd = np.argmin(distanceFromStart) # linear index of new current node
    current_node = np.unravel_index(np.argmin(distanceFromStart), distanceFromStart.shape) # (i,j) tuple
    
    if (current_node in possibleGoalNodes):
        goal_node = current_node
        break
    
    # Update map: mark current node as visited
    state_map[current_node] = 3
    
    # Visit each neighbor of the current node and update the map, distances, and parent tables
    iVec = np.array([current_node[0] - 1, current_node[0], current_node[0] + 1, current_node[0]])
    jVec = np.array([current_node[1], current_node[1] + 1, current_node[1], current_node[1] - 1])
    
    neighbor_nodes = [tuple([iVec[i],jVec[i]]) for i in range(4)]
    
    for neighbor in neighbor_nodes:
            
        # Skip node if out of range
        if neighbor[0] < 0 or neighbor[0] > state_map.shape[0] - 1:
            continue
                
        if neighbor[1] < 0 or neighbor[1] > state_map.shape[1] - 1:
            continue
                
        # Skip node if obstacle, already visited, or start node
        if state_map[neighbor] == 2 or state_map[neighbor] == 3 or neighbor == start_node:
            continue
                
        if state_map[neighbor] == 4:
            # Check if the neighbor cell has been visited yet
            if distanceFromStart[neighbor] > distanceFromStart[current_node] + 1:
                distanceFromStart[neighbor] = distanceFromStart[current_node] + 1
                parent[neighbor] = current_linInd
        else:
            state_map[neighbor] = 4 # Add node to be considered
            distanceFromStart[neighbor] = distanceFromStart[current_node] + 1
            parent[neighbor] = current_linInd
     
    # Remove this node from further consideration
    distanceFromStart[current_node] = np.inf


SyntaxError: invalid syntax (<ipython-input-15-6398a25ee49b>, line 6)

In [ ]:
min_dist = np.min(distanceFromStart)
min_dist

In [ ]:
current_linInd = np.argmin(distanceFromStart) # linear index of new current node
current_linInd

In [ ]:
np.argmin(distanceFromStart)

In [ ]:
distanceFromStart